# Session 4
1. Object Detection 
    - Haar cascade
    - Yolo
2. Object tracking techniques


# Object Detection

## What is the Object Detection? 
<img src = "https://deeplobe.ai/wp-content/uploads/2023/06/Object-detection-Real-world-applications-and-benefits.png" alt="drawing" style="width:500px;"/>

## Object Detection
1. **Recognation** 
2. **loclization** 

## Applications of Object Detection 
1. Object Tracking
2. Self- Driving car
3. Anamoly Detection 


## Haar Classifier - Overview



### Haar Features in images 
    - Efficient Calculation 

###  Traning the Classifier with images
    - Both positive and negative samples

### Selects features during training to create strong classifiers 

### Weak and Stronf Classifiers 
    - Adaboost to make it more efficient and having less features 

### Haar - Features 

<img src= "https://miro.medium.com/v2/resize:fit:1400/1*JhFCP1CjF7fRYt9pLldMsw.jpeg">

- Each feature is a single value obtained by subtracting sum of pixels under the white rectangle from sum of pixels under the black rectangle 
- Specific feature depends on wgat we want to detect 
- How Can we select the best features 
     - Adaboost- finds the best features ? 
     - is used to find the best features and reduce them 

<img src = "https://miro.medium.com/v2/resize:fit:828/format:webp/1*MDkpzf_3JidDLHhR09yawA.png"/>

<img src = "https://miro.medium.com/v2/resize:fit:828/format:webp/1*gl4JHntNPHQt1G7txpiRMA.png"/>

Haar features are similar to these convolution kernels which are used to detect the presence of that feature in the given image.

#### Detect edges using convolution kernels:

Given an input image and convolution kernel, we place kernel to a corner and do convolution multiplication shifting the kernels.



<img src = "https://miro.medium.com/v2/resize:fit:640/0*Ms94ZOyysSZ7c6A8.gif"/>

This method is used to detect different types of edges using different kernels.

A Haar-Feature is just like a kernel in CNN, except that in a CNN, the values of the kernel are determined by training, while a Haar-Feature is manually determined.

<img src = "https://miro.medium.com/v2/resize:fit:640/format:webp/0*g_FpO-ZYVFWiedDF.png"/>

Here are some Haar-Features. The first two are “edge features”, used to detect edges. The third is a “line feature”, while the fourth is a “four rectangle feature”, most likely used to detect a slanted line.

In [4]:
import cv2 as cv
import numpy as np 
import matplotlib.pyplot as plt 

In [2]:
# Load the pre-trained Haar cascade classifier
face_cascade = cv.CascadeClassifier('./HaarCasscade/haarcascade_frontalface_default.xml')

In [3]:
# Load the image
image = cv.imread('img/Faces.jpg')

In [4]:
# Convert the image to grayscale
gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

[detectMultiScale(scaleFactor , minNeighbors , minSize)](https://docs.opencv.org/3.4/d1/de5/classcv_1_1CascadeClassifier.html)

Parameters in detectMultiScale

- `scaleFactor` – This tells how much the object’s size is reduced in each image.
- `minNeighbors` – This parameter tells how many neighbours each rectangle candidate should consider.
- `minSize` — This signifies the minimum possible size of an object to be detected. An object smaller than minSize would be ignored.

Tuning Cascade Classifiers
ourClassifier.detectMultiScale(input image, Scale Factor , Min Neighbors)

    - Scale Factor
Specifies how much we reduce the image size each time we scale. E.g. in face detection we typically use 1.3. This means we reduce the image by 30% each time it’s scaled. Smaller values, like 1.05 will take longer to compute, but will increase the rate of detection.

    - Min Neighbors
Specifies the number of neighbors each potential window should have in order to consider it a positive detection. Typically set between 3-6. It acts as sensitivity setting, low values will sometimes detect multiples faces over a single face. High values will ensure less false positives, but you may miss some faces.

In [5]:
# Perform face detection
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

In [6]:
# Draw bounding boxes around the detected faces
for (x, y, w, h) in faces:
    cv.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)

In [7]:
# Display the image with bounding boxes
cv.imshow('Face Detection', image)
cv.waitKey(0)
cv.destroyAllWindows()

# Yolo (You only look once)

https://www.datacamp.com/blog/yolo-object-detection-explained

In [9]:
def get_output_layers(net):
    
    layer_names = net.getLayerNames()
    try:
        output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    except:
        output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    return output_layers


def draw_prediction(img, class_id, confidence, x, y, x_plus_w, y_plus_h):

    label = str(classes[class_id])

    color = COLORS[class_id]

    cv.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)

    cv.putText(img, label, (x-10,y-10), cv.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)


In [10]:
image = cv.imread("img/dog.jpg")

Width = image.shape[1]
Height = image.shape[0]
scale = 0.00392

classes = None

with open("Yolo/yolov3.txt", 'r') as f:
    classes = [line.strip() for line in f.readlines()]

COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

net = cv.dnn.readNet("Yolo/yolov3.weights", "Yolo/yolov3.cfg")

blob = cv.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)

net.setInput(blob)

outs = net.forward(get_output_layers(net))

class_ids = []
confidences = []
boxes = []
conf_threshold = 0.5
nms_threshold = 0.4


for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            center_x = int(detection[0] * Width)
            center_y = int(detection[1] * Height)
            w = int(detection[2] * Width)
            h = int(detection[3] * Height)
            x = center_x - w / 2
            y = center_y - h / 2
            class_ids.append(class_id)
            confidences.append(float(confidence))
            boxes.append([x, y, w, h])


indices = cv.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

for i in indices:
    try:
        box = boxes[i]
    except:
        i = i[0]
        box = boxes[i]
    
    x = box[0]
    y = box[1]
    w = box[2]
    h = box[3]
    draw_prediction(image, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))

cv.imshow("object detection", image)
cv.waitKey()
    
cv.imwrite("object-detection.jpg", image)
cv.destroyAllWindows()

## Object tracking techniques


In [9]:
import cv2
import statistics
cap=cv2.VideoCapture("Videos/video.mp4")
f=[]
success, img = cap.read()
tracker=cv2.legacy.TrackerMOSSE_create()
bbox=cv2.selectROI("output",img,False)
tracker.init(img ,bbox)
def draw(img,bbox):
    x,y,w,h=int(bbox[0]),int(bbox[1]),int(bbox[2]),int(bbox[3])
    cv2.rectangle(img,(x,y),((x+w),(y+h)),(0,255,255),3,1)
while True:
    success,img=cap.read()
    sucess,bbox=tracker.update(img)
    if sucess:
        draw(img,bbox)
    else:
        cv2.putText(img,"lost",(75,55),cv2.FONT_HERSHEY_SIMPLEX,0.7,(255,255,0),2)
    cv2.imshow("output",img)
    k=cv2.waitKey(1)
    if k==27:
        break
cv2.destroyAllWindows()

In [7]:
tracker = cv2.TrackerKCF_create()
video = cv2.VideoCapture("Videos/video.mp4")
ok,frame=video.read()
bbox=cv2.selectROI("output",frame,False)
ok = tracker.init(frame,bbox)
while True:
   ok,frame=video.read()
   if not ok:
        break
   ok,bbox=tracker.update(frame)
   if ok:
        (x,y,w,h)=[int(v) for v in bbox]
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2,1)
   else:
        cv2.putText(frame,'Error',(100,0),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
   cv2.imshow('Tracking',frame)
   if cv2.waitKey(1) & 0XFF==27:
        break
cv2.destroyAllWindows()

In [12]:
face_classifier = cv2.CascadeClassifier('./HaarCasscade/haarcascade_frontalface_default.xml')

video_capture = cv2.VideoCapture(0)

In [13]:
def detect_bounding_box(vid):
    gray_image = cv2.cvtColor(vid, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray_image, 1.1, 5, minSize=(40, 40))
    for (x, y, w, h) in faces:
        cv2.rectangle(vid, (x, y), (x + w, y + h), (0, 255, 0), 4)
    return faces

In [14]:
while True:

    result, video_frame = video_capture.read()  # read frames from the video
    if result is False:
        break  # terminate the loop if the frame is not read successfully

    faces = detect_bounding_box(
        video_frame
    )  # apply the function we created to the video frame

    cv2.imshow(
        "My Face Detection Project", video_frame
    )  # display the processed frame in a window named "My Face Detection Project"

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

video_capture.release()
cv2.destroyAllWindows()

## Good REF :
1. https://www.datacamp.com/blog/yolo-object-detection-explained
2. https://www.datacamp.com/tutorial/face-detection-python-opencv